此脚本用于Grep命令（grep -aiE "'03046.*2022-09-" *.txt -B1 | grep -aiE "^T.*2022-" > status.csv）之后

第一步将报警信息附加到状态变化信息之后

第二步插值，在状态变化之前3s插入上个变化的信息，如果两个状态变化跨越自然小时，则在整点前后各插入上个状态变化的信息

In [1]:
import pandas as pd
#import os
#import datetime
#import tkinter.filedialog as tk
#import tkinter

In [2]:
dfv = pd.read_excel(r"D:\Sync_ColorCloud\LogAnalysis\AllWerfenChinaTop\downTimeByError - 副本\Data\fseVisit.xlsx",
    usecols=["Model","Engineer (Primary)","Visit Date","Arr.Time","Exit time","Serial Number","Serial Number HemoCell"])
dfv.head()

,Engineer (Primary),Model,Serial Number,Visit Date,Arr.Time,Exit time,Serial Number HemoCell
0,ZAPE000079,2800-35,20040561,2022-09-15,16:00:00,17:30:00,167000031851893
1,ZAPE000079,2800-30J,16110157J,2022-09-22,14:20:00,16:00:00,167000031806916
2,ZAPE000079,2800-35,21080847,2022-09-27,14:00:00,16:15:00,16700318018-160
3,ZAPE000087,2800-30,18060322,2022-09-01,13:00:00,18:30:00,167000031849745
4,ZAPE000087,605-00,167000039088478,2022-09-02,17:10:00,21:10:00,NaN


In [3]:
rename_dic = {
    "Engineer (Primary)":"fse",
    "Model":"model",
    "Serial Number":"sn",
    "Visit Date":"date",
    "Arr.Time":"arrTime",
    "Exit time":"extTime",
    "Serial Number HemoCell":"fsn"}
dfv.rename(columns=rename_dic,inplace=True)
dfv.head()

,fse,model,sn,date,arrTime,extTime,fsn
0,ZAPE000079,2800-35,20040561,2022-09-15,16:00:00,17:30:00,167000031851893
1,ZAPE000079,2800-30J,16110157J,2022-09-22,14:20:00,16:00:00,167000031806916
2,ZAPE000079,2800-35,21080847,2022-09-27,14:00:00,16:15:00,16700318018-160
3,ZAPE000087,2800-30,18060322,2022-09-01,13:00:00,18:30:00,167000031849745
4,ZAPE000087,605-00,167000039088478,2022-09-02,17:10:00,21:10:00,NaN


In [4]:
def add_fsn(row):
    if row.model == "605-00":
        result = row["sn"]
    else:
        result = row["fsn"]
    return result

In [5]:
dfv["fsn"] = dfv.apply(add_fsn,axis=1)
dfv.head()

,fse,model,sn,date,arrTime,extTime,fsn
0,ZAPE000079,2800-35,20040561,2022-09-15,16:00:00,17:30:00,167000031851893
1,ZAPE000079,2800-30J,16110157J,2022-09-22,14:20:00,16:00:00,167000031806916
2,ZAPE000079,2800-35,21080847,2022-09-27,14:00:00,16:15:00,16700318018-160
3,ZAPE000087,2800-30,18060322,2022-09-01,13:00:00,18:30:00,167000031849745
4,ZAPE000087,605-00,167000039088478,2022-09-02,17:10:00,21:10:00,167000039088478


In [6]:
dfv.drop(["model","sn"],inplace=True, axis=1)
dfv.head()

,fse,date,arrTime,extTime,fsn
0,ZAPE000079,2022-09-15,16:00:00,17:30:00,167000031851893
1,ZAPE000079,2022-09-22,14:20:00,16:00:00,167000031806916
2,ZAPE000079,2022-09-27,14:00:00,16:15:00,16700318018-160
3,ZAPE000087,2022-09-01,13:00:00,18:30:00,167000031849745
4,ZAPE000087,2022-09-02,17:10:00,21:10:00,167000039088478


In [7]:
dfv.dropna(inplace=True)

In [8]:
dfv["arrTime"] = pd.to_datetime(dfv.date.astype("str") + " " + dfv.arrTime.astype("str"))
dfv["extTime"] = pd.to_datetime(dfv.date.astype("str") + " " + dfv.extTime.astype("str"))
dfv.head()

,fse,date,arrTime,extTime,fsn
0,ZAPE000079,2022-09-15,2022-09-15 16:00:00,2022-09-15 17:30:00,167000031851893
1,ZAPE000079,2022-09-22,2022-09-22 14:20:00,2022-09-22 16:00:00,167000031806916
2,ZAPE000079,2022-09-27,2022-09-27 14:00:00,2022-09-27 16:15:00,16700318018-160
3,ZAPE000087,2022-09-01,2022-09-01 13:00:00,2022-09-01 18:30:00,167000031849745
4,ZAPE000087,2022-09-02,2022-09-02 17:10:00,2022-09-02 21:10:00,167000039088478


In [9]:
dfv.drop("date",inplace=True,axis=1)
dfv.head()

,fse,arrTime,extTime,fsn
0,ZAPE000079,2022-09-15 16:00:00,2022-09-15 17:30:00,167000031851893
1,ZAPE000079,2022-09-22 14:20:00,2022-09-22 16:00:00,167000031806916
2,ZAPE000079,2022-09-27 14:00:00,2022-09-27 16:15:00,16700318018-160
3,ZAPE000087,2022-09-01 13:00:00,2022-09-01 18:30:00,167000031849745
4,ZAPE000087,2022-09-02 17:10:00,2022-09-02 21:10:00,167000039088478


In [10]:
dfv = dfv.melt(("fse","fsn"),var_name="visitType",value_name="time")

In [11]:
dfv["visitType"] = dfv["visitType"].map({"arrTime":1,"extTime":-1})
dfv

,fse,fsn,visitType,time
0,ZAPE000079,167000031851893,1,2022-09-15 16:00:00
1,ZAPE000079,167000031806916,1,2022-09-22 14:20:00
2,ZAPE000079,16700318018-160,1,2022-09-27 14:00:00
3,ZAPE000087,167000031849745,1,2022-09-01 13:00:00
4,ZAPE000087,167000039088478,1,2022-09-02 17:10:00
...,...,...,...,...
953,ZAPE000779,167000031881061,-1,2022-10-27 12:30:00
954,ZAPE000779,167000031881061,-1,2022-11-14 20:20:00
955,ZAPE000779,167000031881061,-1,2022-11-15 18:55:00
956,ZAPE000779,167000031881061,-1,2022-11-17 21:20:00


In [12]:
def get_hc(father_sn):
    if "-" in father_sn:
        return "H" + father_sn[-3:]
    else:
        return "H0" + father_sn[-2:]

In [13]:
dfv["fsn"] = dfv["fsn"].map(get_hc)

In [14]:
df_las = pd.read_excel(r"D:\Sync_ColorCloud\LogAnalysis\AllWerfenChinaTop\downTimeByError - 副本\Data\toplas.xlsx")

In [15]:
dfv = pd.merge(dfv,df_las,how="left",left_on="fsn",right_on="HC")

In [16]:
def sn_tolist(snlist):
    return snlist.split("_")


In [17]:
dfv["toplist"] = dfv["toplist"].map(sn_tolist)

In [18]:
dfv = dfv.explode("toplist")
dfv.drop(["fsn","HC"],axis=1,inplace=True)

In [19]:
dfv.rename(columns={"visitType":"visitValue","time":"dt","toplist":"sn"},inplace=True)
dfv = dfv.loc[:,["sn","dt","fse","visitValue"]]

In [20]:
visit_dic = dict(list(dfv.groupby("sn")))

In [21]:
logpath = r"D:\Sync_ColorCloud\LogAnalysis\AllWerfenChinaTop\downTimeByError - 副本\Data\GeneralLogs\status.csv"

In [22]:
df1 = pd.read_csv(logpath, sep="\t", encoding="utf_8_sig", header=None, parse_dates=[2])
df1.head()

,0,1,2,3,4,5,6,7,8,9
0,T14060203.txt-'03180,INFORMATION,2022-10-01 07:50:53.520,Setup,NaN,LAS parameters modified.,NaN,0,0,255
1,T14060203.txt:'03046,INFORMATION,2022-10-01 07:51:23.470,Analyzer,NaN,Analyzer Status changed from Not connected to ...,NaN,0,0,255
2,T14060203.txt-'03106,INFORMATION,2022-10-01 07:56:46.577,Other,0,Successful log in by user: 0.,NaN,0,0,255
3,T14060203.txt:'03046,INFORMATION,2022-10-01 07:56:48.667,Analyzer,0,Analyzer Status changed from Emergency stop to...,NaN,0,0,255
4,T14060203.txt:'03046,INFORMATION,2022-10-01 07:57:24.130,Analyzer,0,Analyzer Status changed from Not connected to ...,NaN,0,0,255


In [23]:
df1.drop([1,3,4,6,7,8,9],inplace=True,axis=1)
df1.rename(columns={0 : "rowInfo", 2 : "dt", 5 : "statusChange"},inplace=True)
df1.head()

,rowInfo,dt,statusChange
0,T14060203.txt-'03180,2022-10-01 07:50:53.520,LAS parameters modified.
1,T14060203.txt:'03046,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...
2,T14060203.txt-'03106,2022-10-01 07:56:46.577,Successful log in by user: 0.
3,T14060203.txt:'03046,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...
4,T14060203.txt:'03046,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...


In [24]:
df1.dropna(inplace=True)

In [25]:
def sn_extract(strs):
    sn = strs.split(".")[0]
    return sn

In [26]:
df1.loc[:, "sn"] = df1.rowInfo.map(sn_extract)
df1.loc[:, "code"] = df1.rowInfo.str.slice(-6,)
df1.drop("rowInfo", inplace=True, axis=1)
df1.head()

,dt,statusChange,sn,code
0,2022-10-01 07:50:53.520,LAS parameters modified.,T14060203,'03180
1,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,T14060203,'03046
2,2022-10-01 07:56:46.577,Successful log in by user: 0.,T14060203,'03106
3,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,T14060203,'03046
4,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,T14060203,'03046


In [27]:
df2 = df1.copy()
df2.index += 1

In [28]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=["","_pre"])
df1 = df1.loc[(df1.sn == df1.sn_pre) & (df1.code == "'03046"), :]
df1.drop(["statusChange_pre", "sn_pre", "code"], inplace=True, axis=1)
df1.reset_index(inplace=True, drop=True)
df1.head()

,dt,statusChange,sn,dt_pre,code_pre
0,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,T14060203,2022-10-01 07:50:53.520,'03180
1,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,T14060203,2022-10-01 07:56:46.577,'03106
2,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,T14060203,2022-10-01 07:56:48.667,'03046
3,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,T14060203,2022-10-01 07:57:24.130,'03046
4,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,T14060203,2022-10-01 07:57:35.107,'03046


In [29]:
df1 = df1.loc[:,["sn","dt","statusChange","dt_pre","code_pre"]]
df1.head()

,sn,dt,statusChange,dt_pre,code_pre
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046


In [30]:
ecg_dic = dict(list(df1.groupby("sn")))

In [31]:
l = []
for sn, df in ecg_dic.items():
    if sn in visit_dic.keys():
        dfc = pd.concat([df,visit_dic[sn]])
        dfc.sort_values(by="dt",inplace=True)
        dfc["statusChange"].fillna(method="ffill",inplace=True)
        dfc["dt_pre"].fillna(method="bfill",inplace=True)
        dfc.dropna(subset=["statusChange","dt_pre"],inplace=True,how="any")
        dfc["fse"].fillna(method="ffill",inplace=True)
        dfc["visitValue"].fillna(0,inplace=True)
        if dfc["visitValue"].sum() < 0:
            dfc["visitValue"] = dfc["visitValue"].cumsum() + 1
        else:
            dfc["visitValue"] = dfc["visitValue"].cumsum()
        dfc["code_pre"].fillna("visit",inplace=True)
        l.append(dfc)
    else:
        l.append(df)

In [32]:
df1 = pd.concat(l)

In [33]:
df1.reset_index(drop=True,inplace=True)

In [34]:
df2 = df1.copy()
df2.drop(["statusChange", "dt_pre", "code_pre"], inplace=True, axis=1)
df1.index += 1
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0
3,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0
4,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,0.0
5,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,0.0


In [35]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True,suffixes=["", "_next"])
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,sn_next,dt_next,fse_next,visitValue_next
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,T14060203,2022-10-01 07:56:48.667,NaN,0.0
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,T14060203,2022-10-01 07:57:24.130,NaN,0.0
3,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,T14060203,2022-10-01 07:57:35.107,NaN,0.0
4,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,0.0,T14060203,2022-10-01 07:59:59.253,NaN,0.0
5,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,0.0,T14060203,2022-10-01 08:00:00.000,ZAPE000721,1.0


In [36]:
df1 = df1.loc[df1.sn == df1.sn_next, :]
df1.drop(["sn_next","fse_next","visitValue_next"], inplace=True, axis=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,dt_next
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,2022-10-01 07:56:48.667
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,2022-10-01 07:57:24.130
3,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,2022-10-01 07:57:35.107
4,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,0.0,2022-10-01 07:59:59.253
5,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,0.0,2022-10-01 08:00:00.000


In [37]:
df1.reset_index(inplace=True, drop=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,dt_next
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,2022-10-01 07:56:48.667
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,2022-10-01 07:57:24.130
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,2022-10-01 07:57:35.107
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,0.0,2022-10-01 07:59:59.253
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,0.0,2022-10-01 08:00:00.000


In [38]:
replace_dic = {
    "开机": "Power up",
    "初始化": "Initializing",
    "维护": "Maintenance",
    "错误": "Error",
    "紧急停止": "Emergency stop",
    "忙": "Busy",
    "诊断": "Diagnostics",
    "准备": "Ready",
    "受控停机": "Controlled stop",
    "未连接": "Not connected",
    "温度调整": "Adjusting thermal",
    "分析仪状态从": "Analyzer Status changed from",
    "变为": "to"
}

def replace_desp(desp):
    if "分析仪状态从" not in desp:
        return desp
    for (cn, en) in replace_dic.items():
        desp = desp.replace(cn, en)
    return desp

In [39]:
df1["statusChange"] = df1["statusChange"].map(replace_desp)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,dt_next
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,2022-10-01 07:56:48.667
1,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,2022-10-01 07:57:24.130
2,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,2022-10-01 07:57:35.107
3,T14060203,2022-10-01 07:57:35.107,Analyzer Status changed from Power up to Initi...,2022-10-01 07:57:24.130,'03046,NaN,0.0,2022-10-01 07:59:59.253
4,T14060203,2022-10-01 07:59:59.253,Analyzer Status changed from Initializing to A...,2022-10-01 07:57:35.107,'03046,NaN,0.0,2022-10-01 08:00:00.000


In [40]:
l = []

In [41]:
def insert_rows(df_row):
    global l
    if df_row["dt_next"] - df_row["dt"] < pd.Timedelta(3,"s"):
        l.append(df_row.to_dict())
    elif df_row.dt_next.strftime("%Y%m%d%H") == df_row["dt"].strftime("%Y%m%d%H"):
        l.append(df_row.to_dict())
        l.append({
            "sn" : df_row.sn,
            "dt" : df_row.dt_next - pd.Timedelta(2,"s"),
            "statusChange" : df_row.statusChange,
            "dt_pre" : df_row.dt_pre,
            "code_pre" : df_row.code_pre,
            "fse" : df_row.fse,
            "visitValue" : df_row.visitValue,
            "dt_next" : df_row.dt_next
        })
    else:
        l.append(df_row.to_dict())
        dt_tuple = df_row["dt"].timetuple()
        dt_next_tuple = df_row.dt_next.timetuple()
        dt_hourEnd = pd.Timestamp(dt_tuple.tm_year,dt_tuple.tm_mon,dt_tuple.tm_mday,dt_tuple.tm_hour) + pd.Timedelta('0 days 00:59:59.9')
        dt_next_hourStart = pd.Timestamp(dt_next_tuple.tm_year,dt_next_tuple.tm_mon,dt_next_tuple.tm_mday,dt_next_tuple.tm_hour)
        while dt_hourEnd < dt_next_hourStart:
            l.append({
                "sn" : df_row.sn,
                "dt" : dt_hourEnd, 
                "statusChange" : df_row.statusChange,
                "dt_pre" : df_row.dt_pre,
                "code_pre" : df_row.code_pre,
                "fse" : df_row.fse,
                "visitValue" : df_row.visitValue,
                "dt_next" : df_row.dt_next
            })
            l.append({
                "sn" : df_row.sn, 
                "dt" : dt_hourEnd + pd.Timedelta('0 days 00:00:0.11') , 
                "statusChange" : df_row.statusChange,
                "dt_pre" : df_row.dt_pre,
                "code_pre" : df_row.code_pre,
                "fse" : df_row.fse,
                "visitValue" : df_row.visitValue,
                "dt_next" : df_row.dt_next
            })
            dt_hourEnd += pd.Timedelta('0 days 01:00:00')
        l.append({
            "sn" : df_row.sn, 
            "dt" : df_row.dt_next - pd.Timedelta(2,"s"), 
            "statusChange" : df_row.statusChange,
            "dt_pre" : df_row.dt_pre,
            "code_pre" : df_row.code_pre,
            "fse" : df_row.fse,
            "visitValue" : df_row.visitValue,
            "dt_next" : df_row.dt_next
        })

In [42]:
df1.apply(insert_rows,axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
532520    None
532521    None
532522    None
532523    None
532524    None
Length: 532525, dtype: object

In [43]:
df1 = pd.DataFrame.from_records(l)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,dt_next
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,2022-10-01 07:56:48.667
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,2022-10-01 07:56:48.667
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,2022-10-01 07:57:24.130
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,2022-10-01 07:57:24.130
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,2022-10-01 07:57:35.107


In [44]:
df1.drop("dt_next", inplace=True, axis=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0


In [45]:
df2 = df1.copy()
df1.index += 1
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0
2,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0
3,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0
4,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0
5,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0


In [46]:
df2.drop(["statusChange","dt_pre","code_pre","fse","visitValue"], inplace=True, axis=True)
df2.head()

,sn,dt
0,T14060203,2022-10-01 07:51:23.470
1,T14060203,2022-10-01 07:56:46.667
2,T14060203,2022-10-01 07:56:48.667
3,T14060203,2022-10-01 07:57:22.130
4,T14060203,2022-10-01 07:57:24.130


In [47]:
df1 = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=["", "_next"])
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,sn_next,dt_next
1,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,T14060203,2022-10-01 07:56:46.667
2,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,T14060203,2022-10-01 07:56:48.667
3,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,T14060203,2022-10-01 07:57:22.130
4,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,T14060203,2022-10-01 07:57:24.130
5,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,T14060203,2022-10-01 07:57:33.107


In [48]:
df1 = df1.loc[df1.sn == df1.sn_next, :]
df1.drop("sn_next", inplace=True, axis=1)
df1.reset_index(drop=True, inplace=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,dt_next
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,2022-10-01 07:56:46.667
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,2022-10-01 07:56:48.667
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,2022-10-01 07:57:22.130
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,2022-10-01 07:57:24.130
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,2022-10-01 07:57:33.107


In [49]:
df1.loc[:,"periodH"] = (df1["dt_next"] - df1["dt"]) / pd.Timedelta(1,"h")
df1.loc[:,"date"] = df1["dt"].dt.strftime("%Y/%m/%d")
df1.loc[:,"weekday"] = df1["dt"].dt.weekday + 1
df1.loc[:,"oClock"] = df1["dt"].dt.hour

In [50]:
df1.drop("dt_next", inplace=True, axis=True)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,periodH,date,weekday,oClock
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,0.089777,2022/10/01,6,7
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,0.000556,2022/10/01,6,7
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,0.009295,2022/10/01,6,7
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,0.000556,2022/10/01,6,7
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,0.002494,2022/10/01,6,7


In [51]:
df1.loc[:, "validTimeDiff"] = (df1.dt - df1.dt_pre) / pd.Timedelta(1,"s")
df1.loc[:, "validTimeDiff"] = df1.validTimeDiff.map(lambda x: "y" if x < 2 else "n")
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,periodH,date,weekday,oClock,validTimeDiff
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,0.089777,2022/10/01,6,7,n
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,0.000556,2022/10/01,6,7,n
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,0.009295,2022/10/01,6,7,n
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,0.000556,2022/10/01,6,7,n
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,0.002494,2022/10/01,6,7,n


In [52]:
def status_extract(strs):
    if " to " in strs:
        status = strs.split(" to ")[1]
        return status[0:-1]
    else:
        return strs

In [53]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406101 entries, 0 to 1406100
Data columns (total 12 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   sn             1406101 non-null  object        
 1   dt             1406101 non-null  datetime64[ns]
 2   statusChange   1406101 non-null  object        
 3   dt_pre         1406101 non-null  datetime64[ns]
 4   code_pre       1406101 non-null  object        
 5   fse            452153 non-null   object        
 6   visitValue     1011446 non-null  float64       
 7   periodH        1406101 non-null  float64       
 8   date           1406101 non-null  object        
 9   weekday        1406101 non-null  int64         
 10  oClock         1406101 non-null  int64         
 11  validTimeDiff  1406101 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(6)
memory usage: 128.7+ MB


In [54]:
df1.loc[:, "status"] = df1.statusChange.map(status_extract)
df1.head()

,sn,dt,statusChange,dt_pre,code_pre,fse,visitValue,periodH,date,weekday,oClock,validTimeDiff,status
0,T14060203,2022-10-01 07:51:23.470,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,0.089777,2022/10/01,6,7,n,Emergency stop
1,T14060203,2022-10-01 07:56:46.667,Analyzer Status changed from Not connected to ...,2022-10-01 07:50:53.520,'03180,NaN,0.0,0.000556,2022/10/01,6,7,n,Emergency stop
2,T14060203,2022-10-01 07:56:48.667,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,0.009295,2022/10/01,6,7,n,Not connected
3,T14060203,2022-10-01 07:57:22.130,Analyzer Status changed from Emergency stop to...,2022-10-01 07:56:46.577,'03106,NaN,0.0,0.000556,2022/10/01,6,7,n,Not connected
4,T14060203,2022-10-01 07:57:24.130,Analyzer Status changed from Not connected to ...,2022-10-01 07:56:48.667,'03046,NaN,0.0,0.002494,2022/10/01,6,7,n,Power up


In [55]:
df1.loc[df1.visitValue == 0, "fse"] = "nobody"
df1.fse.fillna("nobody",inplace=True)
df1.visitValue.fillna(0,inplace=True)

In [56]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1406101 entries, 0 to 1406100
Data columns (total 13 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   sn             1406101 non-null  object        
 1   dt             1406101 non-null  datetime64[ns]
 2   statusChange   1406101 non-null  object        
 3   dt_pre         1406101 non-null  datetime64[ns]
 4   code_pre       1406101 non-null  object        
 5   fse            1406101 non-null  object        
 6   visitValue     1406101 non-null  float64       
 7   periodH        1406101 non-null  float64       
 8   date           1406101 non-null  object        
 9   weekday        1406101 non-null  int64         
 10  oClock         1406101 non-null  int64         
 11  validTimeDiff  1406101 non-null  object        
 12  status         1406101 non-null  object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(7)
memory usage: 139.5+ MB


In [57]:
df1.to_csv("status_ecg.csv")